In [1]:
pip install tensorflow-gpu

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for tensorflow-gpu
Failed to build tensorflow-gpu
  Running setup.py install for tensorflow-gpu: started
  Running setup.py install for tensorflow-gpu: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [18 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\Piya\AppData\Local\Temp\pip-install-dcn2cy16\tensorflow-gpu_97e74001458c4e728eb09e8a852b8cdc\setup.py", line 37, in <module>
          raise Exception(TF_REMOVAL_WARNING)
      Exception:
      
      The "tensorflow-gpu" package has been removed!
      
      Please install "tensorflow" instead.
      
      Other than the name, the two packages have been identical
      since TensorFlow 2.1, or roughly since Sep 2019. For more
      information, see: pypi.org/project/tensorflow-gpu
      
      
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tensorflow-gpu
  error: subprocess-ex

In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[]


In [3]:
import tensorflow as tf

print("TensorFlow Version:", tf.__version__)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# List physical devices
physical_devices = tf.config.list_physical_devices('GPU')
print("Physical GPUs:", physical_devices)

# Set memory growth (optional)
for gpu in physical_devices:
    tf.config.experimental.set_memory_growth(gpu, True)

TensorFlow Version: 2.18.0
Num GPUs Available:  0
Physical GPUs: []


In [4]:
pip install mtcnn

     ---------------------------------------- 1.9/1.9 MB 1.1 MB/s eta 0:00:00
     -------------------------------------- 99.8/99.8 kB 952.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import cv2

# Initialize the Haar Cascade face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def is_face_detected(image):
    # Convert PIL image to NumPy array
    image_array = img_to_array(image)
    # Convert RGB image to grayscale
    image_gray = cv2.cvtColor(image_array, cv2.COLOR_RGB2GRAY)
    # Detect faces
    faces = face_cascade.detectMultiScale(image_gray, scaleFactor=1.1, minNeighbors=5)
    # Return True if at least one face is detected
    return len(faces) > 0

# Path to your images
dataset_path = r"C:\Users\Piya\Desktop\My Codes\python code\hair_length_pred\utkface_aligned_cropped\crop_part1"

# Data preparation
images = []
ages = []

for filename in os.listdir(dataset_path):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        age = int(filename.split('_')[0])
        img_path = os.path.join(dataset_path, filename)
        # Load image
        img = load_img(img_path, target_size=(224, 224))
        # Check if the image contains a detectable face
        if is_face_detected(img):
            img_array = img_to_array(img)
            img_array = preprocess_input(img_array)
            images.append(img_array)
            ages.append(age)
        else:
            print(f"Skipping {filename}: No face detected.")

# Convert lists to arrays
X = np.array(images)
y = np.array(ages)

# Check if we have data after filtering
if len(X) == 0:
    raise ValueError("No images with detectable faces found in the dataset.")

# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Build model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1)(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=['mae'])

# Train model
history = model.fit(X_train, y_train,
              validation_data=(X_val, y_val),
              epochs=10,
              batch_size=16)

# Save the model
model.save("age_pred_1.h5")

In [11]:
import cv2
from tensorflow.keras.models import load_model
age_detection_model = load_model(r"C:\Users\Piya\Desktop\My Codes\python code\hair_length_pred\age_pred_1.h5")
def preprocess_image(image, target_size):
    image = cv2.resize(image, target_size)
    image = image / 255.0  
    image = np.expand_dims(image, axis=0)
    return image
def predict_age(image):
    try:
        processed_image = preprocess_image(image, target_size=(224, 224)) 
        predicted_age = age_detection_model.predict(processed_image)
        age = int(predicted_age[0][0])
        return age
    except Exception as e:
        return None

file_path = r"C:\Users\Piya\Desktop\My Codes\python code\hair_length_pred\image to test\Chris_hemsjpg.jpg"    
image = cv2.imread(file_path)    
print("Age: ",predict_age(image))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Age:  30
